In [140]:
%load_ext autoreload
%autoreload 1

import sklearn
import sklearn.datasets
import sklearn.ensemble
from sklearn.compose import ColumnTransformer
import numpy as np
%aimport lime
%aimport lime.lime_tabular
%aimport lime.lime_base

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)

labels = data[:,14]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
data = data[:,:-1]

In [83]:
categorical_features = [1,3,5,6,7,8,9,13]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_

data = data.astype(float)    

In [84]:
categorical_names

{1: array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
        'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
       dtype='<U26'),
 3: array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype='<U26'),
 5: array(['Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
        'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'],
       dtype='<U26'),
 6: array(['?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair',
        'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners',
        'Machine-op-inspct', 'Other-service', 'Priv-house-serv',
        'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support',
        'Transport-moving'], dtype='<U26'),
 7: array(['Husband', 'Not-in-family', 'Other-relative', 'Own-child',
        'Unmarried', 'Wife'], dtype='<U26'),
 8: array(['A

In [85]:
# encoding with new sklearn API
steps = [('OHE', sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'), categorical_features)]
encoder = ColumnTransformer(steps, remainder='passthrough')

In [87]:
np.random.seed(1)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)
encoder.fit(data)
encoded_train = encoder.transform(train)
encoded_test = encoder.transform(test)

In [90]:
import xgboost
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5)
gbtree.fit(encoded_train, labels_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [91]:
sklearn.metrics.accuracy_score(labels_test, gbtree.predict(encoder.transform(test)))

0.8678028558268079

In [157]:
# predict_fn = lambda x: gbtree.predict_proba(x).astype(float)
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x.reshape(1, -1))).astype(float)

In [128]:
explainer = lime.lime_tabular.LimeTabularExplainer(train,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

In [109]:
encoded_test[1653].shape

(1, 108)

In [149]:
print(test[1653].reshape(1, -1).shape)
encoder.transform(test[1653].reshape(1, -1)).shape

# encoder.transform(test[1653])

(1, 14)


(1, 108)

In [162]:
predict_fn(test[1653])

array([[2.31266022e-05, 9.99976873e-01]])

In [163]:
# NOTE: I've added some debugging lines here so line number in trace don't match 1-1 with repo
np.random.seed(1)
explainer.explain_instance(test[1653], predict_fn, num_features=5)
# exp.show_in_notebook(show_all=False)

>>>> mamapapa (5000, 1) (1,)


/Users/muhammadali/Library/Python/3.7/lib/python/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [5000, 1]

In [165]:
test.shape

(6513, 14)